## Text Extraction with Whisper

In this notebook, we will extract information from video/audio files with [Whipser model](https://github.com/openai/whisper). Be leveraging multilingual support, we can extract tanscripts from videos files mixed different languages, even for one video file with different languanges. We provide the following options for whisper inference:
- Batch inference with SageMaker Processing job, we can process massive data and store them into vector database for RAG solution.
- Real-time inference with SageMaker Endpoint, we can leverage it to do summarizaton or QA with a short video/audio file (less than 6MB).

In [2]:
!pip install -U sagemaker -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.27.153 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.6.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.1.1 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.

## Set up

In [3]:
from sagemaker.huggingface import HuggingFaceProcessor
from sagemaker import get_execution_role
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.huggingface import HuggingFaceModel
import sagemaker
import boto3
import json

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.session.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/rag_video"
s3_input = f"s3://{bucket}/{prefix}/raw_data" # Directory for video files
s3_output_clips = f"s3://{bucket}/{prefix}/clips" # Directory for video clips
s3_output_transcript = f"s3://{bucket}/{prefix}/transcript" # Directory for transcripts

In [4]:
%store s3_output_transcript
%store s3_output_clips

Stored 's3_output_transcript' (str)
Stored 's3_output_clips' (str)


## Upload test data to S3 bucket

In [ ]:
!aws s3 cp test_video.mp4 {s3_input}/
!aws s3 cp test_audio.mp3 {s3_input}/

## Batch inference with SageMaker Processing

In [9]:
hfp = HuggingFaceProcessor(
    role=get_execution_role(), 
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    transformers_version='4.28.1',
    pytorch_version='2.0.0', 
    base_job_name='frameworkprocessor-hf',
    py_version="py310"
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [10]:
hfp.run(
    code='preprocessing.py',
    source_dir="data_preparation",
    inputs=[
        ProcessingInput(source=s3_input, destination="/opt/ml/processing/input")
    ], 
    outputs=[
        ProcessingOutput(source='/opt/ml/processing/output_clips', destination=s3_output_clips),
        ProcessingOutput(source='/opt/ml/processing/transcripts', destination=s3_output_transcript),
    ],
    arguments=[
        "--whisper-model", "whisper-large-v2",
        "--target-language", "en",
        "--sentence-embedding-model", "all-mpnet-base-v2"
    ]
)

INFO:sagemaker.processing:Uploaded data_preparation to s3://sagemaker-us-east-1-631450739534/frameworkprocessor-hf-2023-07-15-06-06-27-603/source/sourcedir.tar.gz


Using provided s3_resource


INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-631450739534/frameworkprocessor-hf-2023-07-15-06-06-27-603/source/runproc.sh
INFO:sagemaker:Creating processing-job with name frameworkprocessor-hf-2023-07-15-06-06-27-603


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dg4u832y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dg4u832y
  Resolved https://github.com/openai/whisper.git to commit b91c907694f96a3fb9da03d4bbdc83fbcd3a40a4
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 38.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup

In [11]:
import os
if not os.path.exists('mytest'):
    os.makedirs('mytest')

## Deploy Whipser model to SageMaker for real-time inference

In [12]:
endpoint_name="whisper-large-v2"
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'openai/whisper-large-v2',
    'HF_TASK':'automatic-speech-recognition',
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version='py39',
    
    env=hub,
    role=role
)

In [13]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1, # number of instances
    instance_type='ml.g5.xlarge' # ec2 instance type
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2023-07-15-06-22-35-247
INFO:sagemaker:Creating endpoint-config with name whisper-large-v2
INFO:sagemaker:Creating endpoint with name whisper-large-v2


-----------!

In [ ]:
client = boto3.client('runtime.sagemaker')
file = "test.webm"
with open(file, "rb") as f:
    data = f.read()

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='audio/x-audio', Body=data)
output = json.loads(response['Body'].read())
print(f"Extracted text from the audio file:\n {output['text']}")